# Thermophysical modeling (TPM) of asteroid Didymos

## Load packages

Install the necessary packages only for the first time.

In [ ]:
# using Pkg
# Pkg.add("SPICE")
# Pkg.add("Downloads")
# Pkg.add("StaticArrays")
# Pkg.add("Rotations")
# Pkg.add("CairoMakie")

# Pkg.add(url="https://github.com/Astroshaper/AsteroidThermoPhysicalModels.jl")

In [ ]:
import AsteroidThermoPhysicalModels
import SPICE

using Downloads
using StaticArrays
using Rotations
using CairoMakie

include("../plot_shape.jl");

## Download necessary files
- SPICE kernels
- Shape models

In [ ]:
##= SPICE kernels =##
paths_kernel = [
    "fk/hera_v10.tf",
    "lsk/naif0012.tls",
    "pck/hera_didymos_v06.tpc",
    "spk/de432s.bsp",
    "spk/didymos_hor_000101_500101_v01.bsp",
    "spk/didymos_gmv_260901_311001_v01.bsp",
]

##= Shape models =##
paths_shape = [
    "g_50677mm_rad_obj_didy_0000n00000_v001.obj",
    "g_08438mm_lgt_obj_dimo_0000n00000_v002.obj",
]

##= Download SPICE kernels =##
for path_kernel in paths_kernel
    url_kernel = "https://s2e2.cosmos.esa.int/bitbucket/projects/SPICE_KERNELS/repos/hera/raw/kernels/$(path_kernel)"
    filepath = joinpath("kernel", path_kernel)
    mkpath(dirname(filepath))
    isfile(filepath) || Downloads.download(url_kernel, filepath)
end

##= Download shape models =##
for path_shape in paths_shape
    url_kernel = "https://s2e2.cosmos.esa.int/bitbucket/projects/SPICE_KERNELS/repos/hera/raw/kernels/dsk/$(path_shape)"
    filepath = joinpath("shape", path_shape)
    mkpath(dirname(filepath))
    isfile(filepath) || Downloads.download(url_kernel, filepath)
end

## Load SPICE kernels

In [ ]:
for path_kernel in paths_kernel
    filepath = joinpath("kernel", path_kernel)
    SPICE.furnsh(filepath)
end

Prepare ephemerides for TPM

In [ ]:
P₁ = SPICE.convrt(2.2593, "hours", "seconds")  # Rotation period of Didymos
P₂ = SPICE.convrt(11.93 , "hours", "seconds")  # Rotation period of Dimorphos

ncycles = 5  # Number of cycles to perform TPM
nsteps_in_cycle = 360  # Number of time steps in one rotation period

et_begin = SPICE.utc2et("2027-02-18T00:00:00")  # Start time of TPM
et_end   = et_begin + P₂ * ncycles  # End time of TPM
et_range = range(et_begin, et_end; length=nsteps_in_cycle*ncycles+1);

In [ ]:
"""
Ephemerides data for input into a binary TPM,
including position vectors and rotation matrices depending on the time steps.

- `time` : Ephemeris times
- `sun1` : Sun's position in the primary's frame (DIDYMOS_FIXED)
- `sun2` : Sun's position in the secondary's frame (DIMORPHOS_FIXED)
- `sec`  : Secondary's position in the primary's frame (DIDYMOS_FIXED)
- `P2S`  : Rotation matrix from primary to secondary frames
- `S2P`  : Rotation matrix from secondary to primary frames
"""
ephem = (
    time = collect(et_range),
    sun1 = [SVector{3}(SPICE.spkpos("SUN"      , et, "DIDYMOS_FIXED"  , "None", "DIDYMOS"  )[1]) * 1000 for et in et_range],
    sun2 = [SVector{3}(SPICE.spkpos("SUN"      , et, "DIMORPHOS_FIXED", "None", "DIMORPHOS")[1]) * 1000 for et in et_range],
    sec  = [SVector{3}(SPICE.spkpos("DIMORPHOS", et, "DIDYMOS_FIXED"  , "None", "DIDYMOS"  )[1]) * 1000 for et in et_range],
    P2S  = [RotMatrix{3}(SPICE.pxform("DIDYMOS_FIXED"  , "DIMORPHOS_FIXED", et)) for et in et_range],
    S2P  = [RotMatrix{3}(SPICE.pxform("DIMORPHOS_FIXED", "DIDYMOS_FIXED"  , et)) for et in et_range],
);

Clear the SPICE kernels

In [ ]:
SPICE.kclear()

# Load shape models
The OBJ format is only supported.

In [ ]:
path_shape1_obj = joinpath("shape", "g_50677mm_rad_obj_didy_0000n00000_v001.obj")
path_shape2_obj = joinpath("shape", "g_08438mm_lgt_obj_dimo_0000n00000_v002.obj")

shape1 = AsteroidThermoPhysicalModels.load_shape_obj(path_shape1_obj; scale=1000, find_visible_facets=true)
shape2 = AsteroidThermoPhysicalModels.load_shape_obj(path_shape2_obj; scale=1000, find_visible_facets=true)

println(shape1)  # Didymos shape model
println(shape2)  # Dimorphos shape model

# TPM setup and execution

Thermal properties of Didymos and Dimorphos [Michel+2016; Naidu+2020]

In [ ]:
k  = 0.125   # Thermal conductivity
ρ  = 2170.0  # Material density
Cₚ = 600.0   # Heat capacity at constant pressure

l₁ = AsteroidThermoPhysicalModels.thermal_skin_depth(P₁, k, ρ, Cₚ)  # Thermal skin depth of Didymos
l₂ = AsteroidThermoPhysicalModels.thermal_skin_depth(P₂, k, ρ, Cₚ)  # Thermal skin depth of Dimorphos
Γ₁ = AsteroidThermoPhysicalModels.thermal_inertia(k, ρ, Cₚ);        # Thermal inertia of Didymos
Γ₂ = AsteroidThermoPhysicalModels.thermal_inertia(k, ρ, Cₚ);        # Thermal inertia of Dimorphos

Thermal properties of Didymos 

In [ ]:
thermo_params1 = AsteroidThermoPhysicalModels.thermoparams(
    P       = P₁,     # Rotation period
    l       = l₁,     # Thermal skin depth
    Γ       = Γ₁,      # Thermal inertia
    A_B     = 0.059,  # Bond albedo
    A_TH    = 0.0,    # Albedo in thermal infrared
    ε       = 0.9,    # Emissivity
    z_max   = 0.6,    # Maximum depth to solve the 1-D heat conduction (Lower boundary)
    Nz      = 61,     # Number of depth grid points to solve the 1-D heat conduction equation
)

Thermal properties of Dimorphos

In [ ]:
thermo_params2 = AsteroidThermoPhysicalModels.thermoparams(
    P       = P₂,
    l       = l₂,
    Γ       = Γ₂,
    A_B     = 0.059,
    A_TH    = 0.0,
    ε       = 0.9,
    z_max   = 0.6,
    Nz      = 61,
)

Create a model for the binary asteroid

In [ ]:
## TPM for Didymos
stpm1 = AsteroidThermoPhysicalModels.SingleTPM(shape1, thermo_params1;
    SELF_SHADOWING = true,  # Enable self-shadowing, i.e., shadowing by local topography
    SELF_HEATING   = true,  # Enable self-heating, i.e., energy exchange between interfacing facets
    SOLVER         = AsteroidThermoPhysicalModels.ForwardEulerSolver(thermo_params1),  # Solver for the 1-D heat conduction equation
    BC_UPPER       = AsteroidThermoPhysicalModels.RadiationBoundaryCondition(),        # Upper boundary condition (surface layer)
    BC_LOWER       = AsteroidThermoPhysicalModels.InsulationBoundaryCondition(),       # Lower boundary condition (bottom layer)
)

## TPM for Dimorphos
stpm2 = AsteroidThermoPhysicalModels.SingleTPM(shape2, thermo_params2;
    SELF_SHADOWING = true,
    SELF_HEATING   = true,
    SOLVER         = AsteroidThermoPhysicalModels.ForwardEulerSolver(thermo_params2),
    BC_UPPER       = AsteroidThermoPhysicalModels.RadiationBoundaryCondition(),
    BC_LOWER       = AsteroidThermoPhysicalModels.InsulationBoundaryCondition(),
)

## Combine them to create a binary TPM
btpm = AsteroidThermoPhysicalModels.BinaryTPM(stpm1, stpm2;
    MUTUAL_SHADOWING = true,   # Enable mutual shadowing, i.e., shadowing by the binary pair (eclipse)
    MUTUAL_HEATING   = false,  # Enable mutual heating, i.e., energy exchange between the binary pair, taking much time for computation
);

In [ ]:
AsteroidThermoPhysicalModels.init_temperature!(btpm, 240);  # Intial temperature [K]

Run TPM

In [ ]:
times_to_save = ephem.time[end-nsteps_in_cycle:end]  # Save temperature during the final rotation
face_ID_pri = [1, 2, 3, 4, 10]  # Face indices to save subsurface temperature of Didymos
face_ID_sec = [1, 2, 3, 4, 20]  # Face indices to save subsurface temperature of Dimorphos

result = AsteroidThermoPhysicalModels.run_TPM!(btpm, ephem, times_to_save, face_ID_pri, face_ID_sec; show_progress=false);

Save TPM result

In [ ]:
dirpath = "./"
AsteroidThermoPhysicalModels.export_TPM_results(dirpath, result)

# Data analysis and visualization

Plot the energy conservation ratio `E_cons` to check computational convergence.

`E_cons` is defined as the ratio of the output energy to the input energy from the entire surface of the asteroid over one rotation.
As the thermal calculation converges, `E_cons` approaches 1.

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1],
    xlabel = "Time [h]",
    ylabel = "E_cons [-]",
)

xs = @. (ephem.time - ephem.time[begin]) / 3600  # Time since the beginning of TPM in unit of hour

hlines!(ax, [1], color=:black, linestyle=:dash)
scatterlines!(ax, xs, result.pri.E_cons, marker=:circle, markercolor=:blue,   label="Didymos")
scatterlines!(ax, xs, result.sec.E_cons, marker=:circle, markercolor=:orange, label="Dimorphos")

axislegend(ax, position=:rb)

display(fig)

In [ ]:
plot_shape(shape1; colorbar_title="Radius [m]")  # Plot Didymos shape

In [ ]:
plot_shape(shape2; colorbar_title="Radius [m]")  # Plot Dimorphos shape

In [ ]:
plot_shape(shape1;
    title          = "Temperature map at Didymos eclipse.",
    colorbar_title = "Temperature [K]",
    intensity      = result.pri.surface_temperature[:, end-126],
    colorscale     = :thermal,
)

In [ ]:
plot_shape(shape2;
    title          = "Temperature map at Dimorphos eclipse.",
    colorbar_title = "Temperature [K]",
    intensity      = result.sec.surface_temperature[:, 70],
    colorscale     = :thermal,
)